In [1]:
import pandas as pd
import math
import cv2
from tqdm import tqdm
import os
import re
from scripts.utils import delete_frames, extract_frames, train_test_split
cwd = os.getcwd()

In [5]:
data_path = "../ground_truth_data/"
directories = ["train_data", "validation_data", "test_data"]
muppet_files = ["Muppets-02-01-01.avi", "Muppets-02-04-04.avi", "Muppets-03-04-03"]

for directory in directories:
    dir_path = os.path.join(data_path, directory)
    try:
        os.mkdir(dir_path)
        print(f"Directory '{dir_path}' created successfully")
    except FileExistsError:
        print(f"Directory '{dir_path}' already exists")

Directory '../ground_truth_data/train_data' already exists
Directory '../ground_truth_data/validation_data' already exists
Directory '../ground_truth_data/test_data' already exists


In [ ]:
extract_frames("../ground_truth_data/Muppets-02-04-04.avi", 15)

In [2]:
delete_frames("../ground_truth_data")        

Removed: ../ground_truth_data/frame232.png
Removed: ../ground_truth_data/frame226.png
Removed: ../ground_truth_data/frame144.png
Removed: ../ground_truth_data/frame150.png
Removed: ../ground_truth_data/frame52.png
Removed: ../ground_truth_data/frame178.png
Removed: ../ground_truth_data/frame46.png
Removed: ../ground_truth_data/frame187.png
Removed: ../ground_truth_data/frame193.png
Removed: ../ground_truth_data/frame91.png
Removed: ../ground_truth_data/frame85.png
Removed: ../ground_truth_data/frame84.png
Removed: ../ground_truth_data/frame90.png
Removed: ../ground_truth_data/frame192.png
Removed: ../ground_truth_data/frame186.png
Removed: ../ground_truth_data/frame47.png
Removed: ../ground_truth_data/frame53.png
Removed: ../ground_truth_data/frame179.png
Removed: ../ground_truth_data/frame151.png
Removed: ../ground_truth_data/frame145.png
Removed: ../ground_truth_data/frame227.png
Removed: ../ground_truth_data/frame233.png
Removed: ../ground_truth_data/frame225.png
Removed: ../ground_